In [1]:
from dqn.agent import CarRacingDQN
import os
import tensorflow as tf
import gym
import _thread
import re
import sys

from car_racing_ellipse import CarRacingEllipse
from car_racing_square import CarRacingSquare
from car_racing_zigzag import CarRacingZigZag
from car_racing_cycle_tracks import CarRacingCycleTracks

from gym.wrappers.monitoring import Monitor

import pickle
import h5py

import numpy as np
from matplotlib import pyplot as plt

from datetime import datetime as dt

test_episodes = 100

# SETTINGS

log_frames = False

record_video = True
track_type = 'Combine'

load_checkpoint = True
if track_type=='Square':
    checkpoint_path = 'data/square_20210608'
elif track_type=='Ellipse':
    checkpoint_path = 'data/ellipse_20210608'
elif track_type=='ZigZag':
    checkpoint_path = 'data/zigzag_20210608'
elif track_type=='Combine':
    #checkpoint_path = '/home/dahlin/Desktop/Car Racing/DQN/data/ref_dqn'
    checkpoint_path = '/home/dahlin/Desktop/Car Racing/DQN/data/checkpoint_20210225_num_frame_stack_1_frame_skip_3/Checkpoints'

video_path = os.path.join(checkpoint_path,'Videos')

model_config = dict(
    min_epsilon=0.1,
    max_negative_rewards=12,
    min_experience_size=int(1e4),
    num_frame_stack=1,
    frame_skip=3,
    train_freq=4,
    batchsize=64,
    epsilon_decay_steps=int(1e5),
    network_update_freq=int(1e3),
    experience_capacity=int(4e4),
    gamma=0.95
)

print(model_config)
########

if track_type == 'Square':
    env = CarRacingSquare()
elif track_type == 'Ellipse':
    env = CarRacingEllipse()
elif track_type == 'ZigZag':
    env = CarRacingZigZag()
elif track_type == 'Combine':
    env = CarRacingCycleTracks()

# tf.reset_default_graph()
dqn_agent = CarRacingDQN(env=env, **model_config)
dqn_agent.build_graph()
sess = tf.InteractiveSession()
dqn_agent.session = sess

saver = tf.train.Saver(max_to_keep=100)

print("loading the latest checkpoint from %s" % checkpoint_path)
ckpt = tf.train.get_checkpoint_state(checkpoint_path)
assert ckpt, "checkpoint path %s not found" % checkpoint_path
global_counter = int(re.findall("-(\d+)$", ckpt.model_checkpoint_path)[0])
saver.restore(sess, ckpt.model_checkpoint_path)
dqn_agent.global_counter = global_counter

dqn_agent.do_training = False

/home/dahlin/Desktop/Car Racing/car_racing_retry/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dahlin/Desktop/Car Racing/car_racing_retry/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dahlin/Desktop/Car Racing/car_racing_retry/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

In [2]:
now = dt.now()
os.path.join(checkpoint_path,'dqn_rollouts_'+now.strftime('%Y%m%d_%H%M')+'.h5')

'/home/dahlin/Desktop/Car Racing/DQN/data/ref_dqn/dqn_rollouts_20210610_1144.h5'

In [2]:
now = dt.now()
#desired_len = 100000
desired_episodes = 10

frames_rollout = []
actions_rollout = []
rewards_rollout = []
num_frames_rollout = []
#step_rewards_rollout = []
#dones_rollout = []

#total_reward, frames_in_episode, actions, frames, step_rewards, dones = dqn_agent.play_episode()
total_reward, frames_in_episode, actions, frames = dqn_agent.play_episode()

# actions_rollout.extend(actions)
# frames_rollout.extend(frames)
# rewards_rollout.append(total_reward)
# num_frames_rollout.append(frames_in_episode)
#step_rewards_rollout.extend(step_rewards)
#dones_rollout.extend(dones)
# print('Actions so far : '+str(len(actions_rollout)))
# print("episode: %d, reward: %f, length: %d, total steps: %d" %
#           (dqn_agent.episode_counter, total_reward, frames_in_episode, dqn_agent.global_counter))

episodes = 0

#while(len(actions_rollout)<desired_len):
while episodes<desired_episodes:
    total_reward, frames_in_episode, actions, frames = dqn_agent.play_episode()
    #total_reward, frames_in_episode, actions, frames, step_rewards, dones = dqn_agent.play_episode()
    actions_rollout.extend(actions)
    frames_rollout.extend(frames)
    rewards_rollout.append(total_reward)
    num_frames_rollout.append(frames_in_episode)
    #step_rewards_rollout.extend(step_rewards)
    #dones_rollout.extend(dones)
    print('Actions so far : '+str(len(actions_rollout)))
    print("episode: %d, reward: %f, length: %d, total steps: %d" %
          (dqn_agent.episode_counter, total_reward, frames_in_episode, dqn_agent.global_counter))
    episodes+=1
    
with h5py.File(os.path.join(checkpoint_path,'dqn_rollouts_'+now.strftime('%Y%m%d_%H%M')+'.h5'),'w') as f:
    f.create_dataset('actions_rollout',data=actions_rollout)
    f.create_dataset('frames_rollout',data=frames_rollout)
    f.create_dataset('rewards_rollout',data=rewards_rollout)
    f.create_dataset('num_frames_rollouts',data=num_frames_rollout)
    #f.create_dataset('step_rewards_rollout',data=step_rewards_rollout)
    #f.create_dataset('dones_rollout',data=dones_rollout)

Track generation: 1420..1777 -> 357-tiles track
Track generation: 1340..1781 -> 441-tiles track
Actions so far : 320
episode: 0, reward: 897.181818, length: 320, total steps: 3924047
Track generation: 1127..1503 -> 376-tiles track
Actions so far : 584
episode: 0, reward: 910.161702, length: 264, total steps: 3924047
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 818
episode: 0, reward: 930.000000, length: 234, total steps: 3924047
Track generation: 1340..1781 -> 441-tiles track
Actions so far : 1138
episode: 0, reward: 897.181818, length: 320, total steps: 3924047
Track generation: 1127..1503 -> 376-tiles track
Actions so far : 1402
episode: 0, reward: 910.161702, length: 264, total steps: 3924047
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 1636
episode: 0, reward: 930.000000, length: 234, total steps: 3924047
Track generation: 1340..1781 -> 441-tiles track
Actions so far : 1956
episode: 0, reward: 897.181818, length: 320, total steps: 3924047
Tra

In [3]:
rollout_dict = {}

with h5py.File(os.path.join(checkpoint_path,'dqn_rollouts_'+now.strftime('%Y%m%d_%H%M')+'.h5'),'r') as f:
    for filekey in f.keys():
        print(filekey)
        print(np.shape(f[filekey][:]))
        rollout_dict[filekey] = f[filekey][:]

actions_rollout
(2774,)
frames_rollout
(2774, 96, 96)
num_frames_rollouts
(10,)
rewards_rollout
(10,)
